In [ ]:
print("Hello World!")

## 1.1 Install dependencies


In [ ]:
!conda install -c apple tensorflow-deps

In [ ]:
!python -m pip install tensorflow-macos tensorflow-metal opencv-python matplotlib

In [ ]:
#upgrading the package
!pip install numpy --upgrade 
!pip install pandas --upgrade 
!pip install matplotlib --upgrade 
!pip install scikit-learn --upgrade 
!pip install scipy --upgrade
!pip install plotly --upgrade 

In [ ]:
#print a list of the all the packages 
!pip list

In [ ]:
#import tensorflow
import tensorflow as tf
import os

In [ ]:
#check for available GPUs
tf.config.list_physical_devices("GPU")

## 1.2 Remove doggy images

Imcompatible images needs to be removed from the mass images obtained from the web


In [ ]:
import cv2
import imghdr
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
#set the image directory
data_dir = 'images_data'

#set the desired image extension
image_ext = ['jpeg','jpg','png','bmp']

In [ ]:
os.listdir(data_dir)

In [ ]:
#loops through the class of images
for image_class in os.listdir(data_dir):
    #loops through all the images in each class
    for image in os.listdir(os.path.join(data_dir,image_class)):
        image_path = os.path.join(data_dir,image_class, image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_ext:
                print(f'Image not in ext list {image_path}')
                os.remove(image_path)
        except Exception as e:
            print(f'Issue with image {image_path}')
            #os.remove(image_path)
                
            

## 1.3 Load the Data

In [ ]:
tf.data.Dataset??

In [ ]:
#loading the data with keras utils
#it reshapes the images and puts them in batches (default batch size =32)
data = tf.keras.utils.image_dataset_from_directory('images_data') #this is still like an iterator

In [ ]:
#convert the data into a numpy iterator
data_iterator = data.as_numpy_iterator()

In [ ]:
#get a batch of images
batch = data_iterator.next()

In [ ]:
batch[0][31, 0, 0,:]

In [ ]:
#plot sample images 
fig, ax = plt.subplots(ncols=5, figsize=(20,20))
for idx, img in enumerate(batch[0][:5]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

## 2 Preprocess the data
### 2.1 Scale the Data

In [ ]:
data = data.map(lambda x,y : (x/255,y))

In [ ]:
batch = data.as_numpy_iterator().next()

In [ ]:
#plot sample images 
fig, ax = plt.subplots(ncols=5, figsize=(20,20))
for idx, img in enumerate(batch[0][:5]):
    ax[idx].imshow(img)
    ax[idx].title.set_text(batch[1][idx])

### 2.2 Split the data


In [ ]:
#check the number of batches
data_length = len(data) 

In [ ]:
train_size = int(data_length *.7)
val_size = int(data_length*.2)
test_size = int(data_length*.1)

In [ ]:
assert train_size + val_size + test_size == data_length

In [ ]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [ ]:
train

## 3. Deep Learning
### 3.1 Build the Deep Learning Model

In [ ]:
#import the needed libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout 

In [ ]:
model = Sequential()
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape =(256,256,3)))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())


model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
#complile the model
model.compile('adam', loss = tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
#model summary
model.summary()

## 3.2 Train

In [ ]:
logdir = 'logs'

In [ ]:
#creating a tensorboard cLLback
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
#train the model
history = model.fit(train, epochs=20, validation_data = val, callbacks=[tensorboard_callback])

In [ ]:
history.history

### 3.3 Plot the Performance

In [ ]:
#plot the training loss and validation loss
fig = plt.figure()
plt.plot(history.history['loss'], color ='blue', label='Training Loss')
plt.plot(history.history['val_loss'], color = 'red', label='Validation Loss')
plt.suptitle('Loss Plot', fontsize=16)
plt.legend()
plt.show()

In [ ]:
#plot the training accuracy and validation accuracy
fig = plt.figure()
plt.plot(history.history['accuracy'], color ='blue', label='Training Accuracy')
plt.plot(history.history['val_accuracy'], color = 'red', label='Validation Accuracy')
plt.suptitle('Accuracy Plot', fontsize=16)
plt.legend()
plt.show()

## 4. Evaluate Performance

### 4.1 Evaluate

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
#instantiating the metrics
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
for batch in test.as_numpy_iterator():
    x, y = batch 
    y_pred = model.predict(x)
    pre.update_state(y, y_pred)
    re.update_state(y, y_pred)
    acc.update_state(y, y_pred)

In [ ]:
print(f'Precision: {pre.result()}, Recall: {re.result()}, Accuracy: {acc.result()}')

In [ ]:
batch = test.as_numpy_iterator().next()
fig, ax = plt.subplots(ncols=5, figsize=(20,20))
for i, image in enumerate(batch[0][:5]):
    ax[i].imshow(image)
    ax[i].title.set_text(batch[1][i])

### 4.2 Test

In [ ]:
#load a random image not seen before by the classifer
img = cv2.imread('Image 3.jpeg')
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [ ]:
#preprocess the test data
img_resized = tf.image.resize(img, (256,256))
plt.imshow(img_resized.numpy().astype(int))

In [ ]:
y_pred = model.predict(np.expand_dims(img_resized/255,0))

In [ ]:
y_pred

In [ ]:
if y_pred < 0.5:
    print('Predicted class is Happy!')
else: print('Predicted class is Sad')

## 5. Save the Model

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save('happy_sad_imageclassificationmodel.h5')

In [ ]:
new_model = load_model('happy_sad_imageclassificationmodel.h5')

In [ ]:
new_model

In [ ]:
!pipreqs